# Lab 7: Learning to Rank

To solve the following problems, we will use the Whoosh search library for Python. The goal is to implement and evaluate a simple search engine.

### pri_cfc.txt

The file `pri_cfc.txt` is composed of several text documents from the CFC collection. It contains one document per line, and the first word of each line is the document ID.

See:

https://bitbucket.org/mchaput/whoosh/wiki/Home. All code excerpts shown here are adapted from the Whoosh documentation.

http://www.dcc.ufmg.br/irbook/cfc.html

`00001 The significance of Pseudomonas aeruginosa infection in the respiratory tract of 9 cystic fibrosis patients have been studied by means of immunoelectrophoretical analysis of patients' sera for the number of precipitins against Pseudomonas aeruginosa and the concentrations of 16 serum proteins.  In addition, the clinical and radiographical status of the lungs have been evaluated using 2 scoring systems.  Precipitins against Pseudomonas aeruginosa were demonstrated in all sera, the maximum number in one serum was 22. The concentrations of 12 of the serum proteins were significantly changed compared with matched control persons.  Notably IgG and IgA were elevated and the "acute phase proteins" were changed, the latter suggesting active tissue damage.  The concentrations of 3 of the acute phase proteins, notably haptoglobin, were correlated to the number of precipitins suggesting that the respiratory tract infection in patients with many precipitins is accompanied by more tissue damage than the infection in patients with few precipitins. The results indicate no protective value of the many precipitins on the tissue of the respiratory tract.
00002 Salivary amylase levels were determined in normal subjects from birth until adult life and in children with conditions sometimes associated with low pancreatic amylase such as malnutrition, coeliac disease and cystic fibrosis.  Mixed saliva was collected under carefully standardised conditions and amylase was measured by the method of Dahlqvist.  There was a wide scatter of values in the 84 normal subjects, but concentrations rose from very low levels at birth to reach adult levels by the age of 6 months to 1 year. Salivary amylase activity rose normally over ten weeks in one premature infant fed milk by gastrostomy.  Thirteen children with coeliac disease and 9 children with cystic fibrosis mostly had normal salivary amylase concentrations.  Six out of 12 malnourished children with jejunal villous atrophy of uncertain aetiology had low levels which rose to normal as recovery began.
00003 This article reports on the possibility of using instrumental neutron activation analysis (INAA) of sodium in nail clippings for diagnosing cystic fibrosis (CF) in children and adults, for detecting heterozygotes and for screening in the neonatal period. Nail clippings from 1322 newborns, 22 CF patients (two of them newborns), 52 healthy controls and 22 heterozygotes were analyzed. The discrimination between CF patients and controls was found to be precise for individuals above one year of age and INAA of nail clippings should be accepted as a diagnostic test for CF after this age.  Heterozygotes could not be detected by the method.  During the first five days of life there is a big overlap between the values from normal newborns and those of CF children, which makes the method invaluable for early screening for CF.`

Create a Python script that indexes all the documents using the Whoosh library. Make sure that you also store the document ID in a separate field.

**Note:** The directory indexdir must already exist, for the index to be created.

In [1]:
from whoosh.index import create_in
from whoosh.fields import *

def index(fname):
    schema = Schema(id = NUMERIC(stored=True), content=TEXT(stored=True))
    ix = create_in("indexdir", schema)
    writer = ix.writer()

    #Read file.
    with open(fname) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        #Store id and content.
        for doc in content:
            doc = doc.split(' ')
            id = int(doc[0])
            content = ' '.join(doc[1:])
            writer.add_document(id=id,content=content)
    writer.commit()
    return ix

    
fname = 'pri_cfc.txt'
ix = index(fname)
n_docs = len(list(ix.searcher().documents()))
print(n_docs)

1239


### pri_queries.txt

The file `pri_queries.txt` contains a set of queries and, for each query, the set of relevant documents.


`What are the effects of calcium on the physical properties of mucus from CF patients? 
139 151 166 311 370 392 439 440 441 454 461 502 503 505 520 522 526 527 533 593 619 737 742 789 827 835 861 875 891 921 922 1175 1185 1222
Can one distinguish between the effects of mucus hypersecretion and infection on the submucosal glands of the respiratory tract in CF?
169 434 454 498 499 592 875
How are salivary glycoproteins from CF patients different from those of normal subjects?
23 40 139 190 221 246 309 311 325 345 347 356 370 374 375 439 440 454 515 520 524 526 527 533 535 560 561 571 584 604 623 633 733 742 854 856 950 967 1144 1161 1172 1175 1196
What is the lipid composition of CF respiratory secretions?
503 538 539 540 553 604 669 711 876`

In [2]:
def read_query(n_docs,queryfile):
    queries = dict()
    #Open and read query file.
    with open(queryfile) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
        print(content[:2])
        
        #Parse query file.
        for i in range(0,len(content),2):
            docs = [0 for k in range(n_docs)]

            #Select relevant documents for each query i.
            for ind in content[i+1].split():
                #print(int(ind)-1)
                docs[int(ind)-1] = 1
            
            queries[content[i]] = docs

    return queries

queryfile = 'pri_queries.txt'
queries = read_query(n_docs,queryfile)
print(queries)

['What are the effects of calcium on the physical properties of mucus from CF patients?', '139 151 166 311 370 392 439 440 441 454 461 502 503 505 520 522 526 527 533 593 619 737 742 789 827 835 861 875 891 921 922 1175 1185 1222']
{'What are the effects of calcium on the physical properties of mucus from CF patients?': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 1 Guiding ranking

The Whoosh search engine provides three different ranking functions: BM25, TF IDF (under a cosine similarity) and Frequency.

Let us recover the pri_cfc collection (files pri_cfc.txt and pri_queries.txt) introduced in earlier labs. Use Whoosh to index it and run queries using one of the provided ranking functions.

Let us create a method for scoring the documents that combines the results from the three scoring functions.

See:
https://whoosh.readthedocs.io/en/latest/api/scoring.html

In [3]:
from whoosh.index import open_dir
from whoosh.qparser import *
from whoosh import scoring

def rank(query_str,weighting=scoring.BM25F(),limit=None):
    ''' Perform a query using the weighting scoring function and obtain the corresponding textual similarity score. '''
    ix = open_dir("indexdir")

    with ix.searcher(weighting=weighting) as searcher:
        query = QueryParser("content", ix.schema, group=OrGroup).parse(query_str)
        results = searcher.search(query,limit=limit)
        query_res = dict()
        for i,r in enumerate(results):
            id = r['id']
            #print(i,results.score(i),r['id'],r['content'],'\n')
            query_res[id] = results.score(i)
        return query_res

### 1.1 Implement a script that performs searches and returns the results ordered by a linear combination of the three textual similarities presented above (BM25, TF IDF and Frequency).

The rank combination formula should be:

*score(q, d) = alpha_1 . bm25(q, d) + alpha_2 . cos(q, d) + alpha_3 . freq(q, d)*

where *d* is the document, *q* is the query, *bm25* is the score obtained using the BM25 ranking function, *cos* is the score obtained using the TF-IDF ranking function, and *freq* is the score obtained using the Frequency ranking function.

Assess how different values for weights *alpha_1*, *alpha_2*, and *alpha_3* impact the Mean Average Precision (MAP) against each individual ranking function used in isolation.

In [4]:
def generate_scores(query):
    '''Generate scores for a given query according to BM25, TF IDF (under a cosine similarity) and Frequency rank functions'''
    bm25 = rank(query,weighting=scoring.BM25F(),limit=None)
    cos = rank(query,weighting=scoring.TF_IDF(),limit=None)
    freq = rank(query,weighting=scoring.Frequency(),limit=None)
    return bm25,cos,freq

In [5]:
def score(n_docs,n_rel,bm25,cos,freq,alpha_1=3,alpha_2=3,alpha_3=1):
    scores = dict()
    #Iterate over all documents in collection.
    for k,v in bm25.items():
        #Rank combination.
        scores[k] = alpha_1*bm25[k] + alpha_2*cos[k] + alpha_3*freq[k]
   
    #Assign 0 if document was not a hit.
    combined_scores = [0 for i in range(n_docs)]
    for key,val in scores.items():
        combined_scores[key-1] = val

    return combined_scores

In [6]:
from sklearn.metrics import average_precision_score

for index,(query, y_true) in enumerate(queries.items()):
    #Rank according using different rank functions. 
    bm25,cos,freq = generate_scores(query)

    #Count the number of relevant documents for a given query.
    n_rel = y_true.count(1)

    #Generate y_pred according to the combined score.
    y_pred = score(n_docs,n_rel,bm25,cos,freq,alpha_1=1,alpha_2=0,alpha_3=0)
    
    print('q = {}'.format(query))
    print('average_precision_score = {} \n'.format(average_precision_score(y_true, y_pred)))
    
    if index == 5:
        break

q = What are the effects of calcium on the physical properties of mucus from CF patients?
average_precision_score = 0.20783755850589009 

q = Can one distinguish between the effects of mucus hypersecretion and infection on the submucosal glands of the respiratory tract in CF?
average_precision_score = 0.1093014749836515 

q = How are salivary glycoproteins from CF patients different from those of normal subjects?
average_precision_score = 0.13565560035812854 

q = What is the lipid composition of CF respiratory secretions?
average_precision_score = 0.04783825081661492 

q = Is CF mucus abnormal?
average_precision_score = 0.26159805816137505 

q = What is the effect of water or other therapeutic agents on the physical properties (viscosity, elasticity) of sputum or bronchial secretions from CF patients?
average_precision_score = 0.19475369380608676 



### 1.2 Pointwise Learning to Rank (L2R) approach

The goal now is to try a more sophisticated approach for combining the ranking functions. To this effect we will use a pointwise Learning to Rank (L2R) approach.

Our approach consists in training a Logistic Regression classifier on the set of queries available in `pri_queries.txt`.

See: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

More specifically, you should:

(a) Create a dataset for training and testing your L2R approach:
* use 50% of the queries for training and 50% for testing (you can vary these percentages if you wish);
* with the training queries, build the training dataset. This dataset should contain, for each (query `q`, document `d`) pair, a set of classification instances with the format: `bm25(q, d); cos(q, d); freq(q, d); r`, where `r = 1` if document `d` is relevant for query `q` and `r = 0` otherwise. You can store this data on a numpy array;
* use the same number of relevant and non-relevant documents for each query.

In [7]:
import random
from copy import deepcopy

def get_train_test_queries(queries,split=0.5):
    '''Generate the train and test splits.'''
    
    n_queries = len(queries.keys())
    train,test = dict(),queries.copy()

    for i in range(int(n_queries*split)):
        key = random.choice(list(test.keys()))
        val = test[key]
        
        train[key] = val
        del test[key]
        
    return train, test

In [8]:
def add_query_doc(dataset,q,d,bm25,cos,freq,r):
    dataset[q] += [{
        'doc': d,
        'bm25': bm25.get(d,0),
        'cos': cos.get(d,0),
        'freq': freq.get(d,0),
        'rel': r
    }]
    return dataset
    
    
def build_dataset(subset,n_docs):

    dataset = dict()
    #Create relevant and non relevant sets for each query.
    for query, y_true in subset.items():

        #Rank according using different rank functions. 
        bm25,cos,freq = generate_scores(query)
        
        dataset[query] = []

        #Number of relevant documents for query.
        n_rel = y_true.count(1)

        for i in range(n_docs):
            #If document is relevant for query.
            if(y_true[i] == 1):
                dataset = add_query_doc(dataset,query,i+1,bm25,cos,freq,y_true[i])

            #If we can still add non relevant documents to the subset.
            elif(n_rel>0):
                n_rel-=1
                dataset = add_query_doc(dataset,query,i+1,bm25,cos,freq,y_true[i])
                
    return dataset

In [9]:
queries_train, queries_test = get_train_test_queries(queries)

train = build_dataset(queries_train,n_docs)
test = build_dataset(queries_test,n_docs)

(b) Use the training dataset to learn the logistic regression classifier:
* the three ranking scores will be your classification features and r the target class.

In [10]:
def generate_features(dataset):
    '''Generate feature vector and the target class for a dataset.'''
    X,y_true = [],[]

    for query,docs in dataset.items():
        for doc in docs:
            X+=[[doc['bm25'],doc['cos'],doc['freq']]]
            y_true+=[doc['rel']]
    return X,y_true

In [11]:
from sklearn.linear_model import LogisticRegression    

X_train,y_train = generate_features(train)

clf = LogisticRegression(
        random_state=0, solver='lbfgs',
        multi_class='multinomial').fit(X_train, y_train)

(c) Execute the queries on the testing set, using the Logistic Regression classifier as your ranking function. Measure: Precision, Recall, and F1 scores for the classifier, and measure the Mean Average Precision (MAP) for the produced ranking.

* to do this, first perform regular searches, using each ranking function in isolation;
* the score of each ranking function will be the classification features and the classifier will return 1 if the document is relevant or 0 if otherwise;
* to order the resulting documents, you should use the probability of the document being relevant. This can be obtained through the predict proba method of the LogisticRegression class.

In [12]:
X_test,y_test = generate_features(test)

In [13]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = clf.predict(X_test)

# average='macro': Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
prf1 = precision_recall_fscore_support(y_test, y_pred, average='macro') 

print('Precision: {}'.format(prf1[0]))
print('Recall: {}'.format(prf1[1]))
print('F1 score: {}'.format(prf1[2]))

Precision: 0.6524441961204626
Recall: 0.6399201596806388
F1 score: 0.6323694986645285


Mean average precision for a set of queries is the mean of the average precision scores for each query. 

In [14]:
from statistics import mean

avg_prec_scores = []

#Compute the average precision score for each query in the test set.
for query in test.keys():
    #Generate features for each query's test set.
    X_test,y_test = generate_features({query:test[query]})
    #Predict scores.
    y_scores = clf.predict_proba(X_test)
    #Append average precision scores to a global list.
    avg_prec_scores += [average_precision_score(y_test, y_scores[:,1])]

print('MAP score: {}'.format(mean(avg_prec_scores)))

MAP score: 0.7524699654331765
